In [5]:
!pip install --upgrade transformers datasets --quiet

from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

# Sample data
support_tickets = [
    {"ticket": "I was charged twice for the same month.", "label": "billing"},
    {"ticket": "My app crashes every time I open it.", "label": "bug report"},
    {"ticket": "I can't log in even after resetting my password.", "label": "account access"},
    {"ticket": "How do I change my subscription plan?", "label": "general inquiry"},
    {"ticket": "The new update has broken the search feature.", "label": "bug report"},
    {"ticket": "Can you add a dark mode feature?", "label": "feature request"},
    {"ticket": "I'm not receiving OTP codes on my phone.", "label": "account access"},
]

TAGS = ["billing", "technical issue", "account access", "feature request", "bug report", "general inquiry"]
label2id = {label: i for i, label in enumerate(TAGS)}
id2label = {i: label for label, i in label2id.items()}

# Tokenization and preprocessing
dataset = Dataset.from_list(support_tickets)
dataset = dataset.map(lambda x: {"label": label2id[x["label"]]})
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = dataset.map(lambda x: tokenizer(x["ticket"], truncation=True, padding="max_length"), batched=True)

# Train-test split
train_dataset = dataset.shuffle(seed=42).select(range(5))
test_dataset = dataset.select(range(5, len(dataset)))

# Model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(TAGS),
    id2label=id2label,
    label2id=label2id
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=5,
    eval_strategy="epoch"
)
import os
os.environ["WANDB_DISABLED"] = "true"
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)


trainer.train()

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/tmp/ipython-input-5-374489683.py:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss
1,No log,1.920227
2,1.861100,1.925657
3,1.861100,1.921807


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=9, training_loss=1.7041683197021484, metrics={'train_runtime': 161.4205, 'train_samples_per_second': 0.093, 'train_steps_per_second': 0.056, 'total_flos': 3946807572480.0, 'train_loss': 1.7041683197021484, 'epoch': 3.0})

In [6]:
import torch
def predict_top3(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1).squeeze()
    top3 = torch.topk(probs, 3)
    return [TAGS[i] for i in top3.indices]

for ticket in support_tickets:
    top_tags = predict_top3(ticket["ticket"])
    print(f"\n📩 Ticket: {ticket['ticket']}")
    print(f"🔮 Top 3 Tags: {top_tags}")
    print(f"🎯 Actual: {ticket['label']}")



📩 Ticket: I was charged twice for the same month.
🔮 Top 3 Tags: ['general inquiry', 'bug report', 'account access']
🎯 Actual: billing

📩 Ticket: My app crashes every time I open it.
🔮 Top 3 Tags: ['bug report', 'account access', 'billing']
🎯 Actual: bug report

📩 Ticket: I can't log in even after resetting my password.
🔮 Top 3 Tags: ['account access', 'bug report', 'feature request']
🎯 Actual: account access

📩 Ticket: How do I change my subscription plan?
🔮 Top 3 Tags: ['general inquiry', 'billing', 'bug report']
🎯 Actual: general inquiry

📩 Ticket: The new update has broken the search feature.
🔮 Top 3 Tags: ['general inquiry', 'bug report', 'account access']
🎯 Actual: bug report

📩 Ticket: Can you add a dark mode feature?
🔮 Top 3 Tags: ['general inquiry', 'bug report', 'account access']
🎯 Actual: feature request

📩 Ticket: I'm not receiving OTP codes on my phone.
🔮 Top 3 Tags: ['general inquiry', 'account access', 'bug report']
🎯 Actual: account access
